# Kaggle Learn
#### Deep Learning Level 1

<br>Source: https://www.kaggle.com/learn/overview
<br>Slide Command: jupyter nbconvert DeepLearningPresentation.ipynb --to slides --post serve
<br><br>

Presentor: Kavi Sekhon


### Topics Dicusssed
- Deep Learning Videos
- Using a Pretrained Model
- Transfer Learning
- Building a Model from Scratch
- Using Dropout

### Videos

- Kaggle Deep Learning 1: https://www.youtube.com/watch?v=wG6rdUURU-w&feature=youtu.be 
<br>
- Kaggle Deep Learning 2: https://www.youtube.com/watch?v=Bqch2ptaAJ8&feature=youtu.be
- Kaggle Deep Learning 3: https://www.youtube.com/watch?v=sDG5tPtsbSA&feature=youtu.be
- Kaggle Deep Learning 4: https://www.youtube.com/watch?v=qeLL23wD2Zo&feature=youtu.be
- Kaggle Deep Learning 5: https://www.youtube.com/watch?v=KysXEztDgeQ&feature=youtu.be
- Kaggle Deep Learning 6: https://www.youtube.com/watch?v=JZBXuINX6E4&feature=youtu.be
- Kaggle Deep Learning 7: https://www.youtube.com/watch?v=zN8z-Y7Tr-8&feature=youtu.be
- Kaggle Deep Learning 8: https://www.youtube.com/watch?v=pPiErCD_EGk&feature=youtu.be

- Tensor Flow and Keras
- What a is a Tensor? 1D, 2D, 3D?
- Grey Images are 2D?
- RBG IMages are 3D?
- Timeseries Data = 4D?
- Video Tensor = 5?

### Using A Pretrained Model 

In [9]:
# Choose images to work with
from os.path import join

# Generating a list of file names
image_dir = 'Images/train/'
img_paths = [join(image_dir, filename) for filename in ['0246f44bb123ce3f91c939861eb97fb7.jpg',
                            '84728e78632c0910a69d33f82e62638c.jpg',
                            '8825e914555803f4c67b26593c9d5aff.jpg',
                            '91a5e8db15bccfb6cfa2df5e8b95ec03.jpg']]

In [10]:
### Function to Read and Prep Images for Modeling
import numpy as np
import tensorflow
import keras

size = 24

import numpy as np
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array

# variable for images height and width
image_size = 224

# read modifies images 
def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
  imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
  img_array = np.array([img_to_array(img) for img in imgs])
  output = preprocess_input(img_array)
  return output

In [11]:
from keras.applications import ResNet50

model = ResNet50(weights='imagenet')
data = read_and_prep_images(img_paths)
predictions =  model.predict(data)

# Fix: Replace resnet50_weights_tf_dim_ordering_tf_kernels.h5 with 'imagenet'

### Viewing Predicitons

In [12]:
import sys

sys.path.append('/kaggle/input/python-utility-code-for-deep-learning-exercise/utils')
from keras.applications.resnet50 import decode_predictions

from IPython.display import Image,display

most_likely_labels = decode_predictions(predictions, top=3)

for i, img_path in enumerate(img_paths):
  #display(Image(img_paths))
  print(most_likely_labels[i])   
  print()

[('n02097209', 'standard_schnauzer', 0.57363278), ('n02097047', 'miniature_schnauzer', 0.42467678), ('n02097130', 'giant_schnauzer', 0.00094619842)]

[('n02092339', 'Weimaraner', 0.99369735), ('n02109047', 'Great_Dane', 0.0013664225), ('n02100236', 'German_short-haired_pointer', 0.001143955)]

[('n02105855', 'Shetland_sheepdog', 0.9486419), ('n02106030', 'collie', 0.044787489), ('n02106166', 'Border_collie', 0.0015023499)]

[('n02110627', 'affenpinscher', 0.60964632), ('n02112706', 'Brabancon_griffon', 0.19211127), ('n02094433', 'Yorkshire_terrier', 0.046045057)]



## Transfer Learning

### Transfer Learning with ImageNet

In [13]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2

model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', 
                          weights='imagenet'))
model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) moedl. It is already trained
model.layers[0].trainable = False

### Compile Model

In [14]:
model.compile(optimizer='sgd', loss='categorical_crossentropy',
            metrics=['accuracy'])

### Fit Model

In [15]:
from keras.preprocessing.image import ImageDataGenerator

image_size = 224

data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = data_generator.flow_from_directory('Data/rural_and_urban_photos/train',
      target_size=(image_size, image_size), batch_size=24, class_mode='categorical')

validation_generator = data_generator.flow_from_directory('Data/rural_and_urban_photos/val',
      target_size=(image_size, image_size), class_mode='categorical')

model.fit_generator(train_generator, 
                    steps_per_epoch=3,
                    validation_data=validation_generator,
                    validation_steps=1)

Found 72 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/1
3/3 [==============================] - 84s 28s/step - loss: 0.7942 - acc: 0.5556 - val_loss: 0.3098 - val_acc: 1.0000


## Data Augmentation

###  Fitting a Model With Data Augmentation

In [1]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

image_size = 224

data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                            horizontal_flip=True, width_shift_range = 0.2,
                                            height_shift_range = 0.2)

train_generator = data_generator_with_aug.flow_from_directory('Data/rural_and_urban_photos/train',
                                                             target_size=(image_size, image_size),
                                                             batch_size=24,
                                                             class_mode='categorical')

data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_generator = data_generator_no_aug.flow_from_directory('Data/rural_and_urban_photos/val',
                                                                target_size=(image_size, image_size),
                                                                class_mode='categorical')

model.fit_generator(train_generator,
                          steps_per_epoch=3,
                          epochs=2,
                          validation_data=validation_generator,
                          validation_steps=1)

Using TensorFlow backend.


Found 72 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


NameError: name 'model' is not defined

## A Deep Learning From Scratch

### Fitting a Model from Scratch

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Dropout

a = pd.read_csv('Data/Digit Recognizer/train.csv')
a.drop('label', axis=1)
a.head()

In [ ]:
img_rows, img_cols = 28, 28
num_classes =  10

def data_prep(raw):
  out_y = keras.utils.to_categorical(raw.label, num_classes)
  
  num_images = raw.shape[0]
  x_as_array = raw.values[:,1:]
  x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
  out_x = x_shaped_array / 255
  return out_x, out_y

train_size = 30000
train_file = 'Data/Digit Recognizer/train.csv'

raw_data = pd.read_csv(train_file)
x, y = data_prep(raw_data)
  
model = Sequential()
model.add(Conv2D(20, kernel_size=(3,3),
                activation='relu',
                input_shape=(img_rows, img_cols, 1)))

model.add(Conv2D(20, kernel_size=(3,3),
                activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer='adam',
             metrics=['accuracy'])

model.fit(x, y,
         batch_size=128,
         epochs=2,
         validation_split=0.2)

## Dropout and Strides for Larger Models

### Fitting a Model with Dropout

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Dropout

a = pd.read_csv('Data/Digit Recognizer/train.csv')
a.drop('label', axis=1)

img_rows, img_cols = 28, 28
num_classes =  10

def data_prep(raw):
  out_y = keras.utils.to_categorical(raw.label, num_classes)
  
  num_images = raw.shape[0]
  x_as_array = raw.values[:,1:]
  x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
  out_x = x_shaped_array / 255
  return out_x, out_y

train_size = 30000
train_file = 'Data/Digit Recognizer/train.csv'

raw_data = pd.read_csv(train_file)
x, y = data_prep(raw_data)
  
model = Sequential()
model.add(Conv2D(30, kernel_size=(3,3),
                activation='relu',
                input_shape=(img_rows, img_cols, 1)))
Dropout(0.5)
model.add(Conv2D(30, kernel_size=(3,3), strides=2,
                activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer='adam',
             metrics=['accuracy'])

model.fit(x, y,
         batch_size=128,
         epochs=2,
         validation_split=0.2)